在风控领域有两种变量编码方式，对实际业务中的分类型变量进行映射得到数值型变量：

one-hot编码

WOE编码（Weight of Evidence）

**one-hot编码**

In [1]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
enc.transform([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]]).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:395: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  check_array(X, dtype=np.int)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='

array([[1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 1., 0.]])

**字符型变量自动化WOE编码**

WOE表示的含义即是"当前分组中响应客户占所有响应客户的比例"和"当前分组中没有响应的客户占所有没有响应客户的比例"的差异。

在机器学习的二分类问题中，IV值（Information Value）主要用来对输入变量进行编码和预测能力评估。特征变量IV值的大小即表示该变量预测能力的强弱。IV 值的取值范围是[0, 正无穷)，如果当前分组中只包含响应客户或者未响应客户时，IV = 正无穷。量化指标含义如下：

小于0.02 : useless for prediction

0.02 to 0.1 : Weak predictor

0.1 to 0.3 : Medium predictor

0.3 to 0.5 : Strong predictor 

大于0.5 ：Suspicious or too good to be true

In [1]:
import math

In [2]:
class charWoe(object):
    def __init__ (self, datasets, dep, weight, vars):
        self.datasets = datasets
        self.devf = datasets.get("dev", "") #训练集
        self.valf = datasets.get("val", "") #测试集
        self.offf = datasets.get("off", "") #跨时间验证集
        self.dep = dep #标签
        self.weight = weight #样本权重
        self.vars = vars #参与建模的特征名
        self.nrows, self.ncols = self.devf.shape #样本数，特征数
 

In [3]:
    def char_woe(self):
        #得到每一类样本的个数
        dic = dict(self.devf.groupby([self.dep]).size())
        good  = dic.get(0, 0) + 1e-10 #加入平滑想使不为0
        bad  = dic.get(1, 0) + 1e-10
        #对每个特征进行遍历
        for cols in self.vars:
            #得到每一个特征值对应的样本数
            data = dict(self.devf[[col,self.dep]].groupby([col,self.dep]).size())
            
            '''当特征值取值超过100个，跳过当前取值
            因为取值过多时，WOE分箱的效率较低，建议对特征进行截断。 
            出现频率过低的特征值统一赋值，放入同一箱内。
            '''
            
            if len(data) > 100:
                print(col, "取值过多")
                continue
            print(col, len(data)) #打印个数
            dic = dict()
            for (k,v) in data.items():
                value, dp = k #特征名，特征取值
                dic.setdefault(value,{}) #如果找不到key 设置一个空字典
                #字典中嵌套字典
                dic[value][int(dp)] = v #v为样本数
            for (k,v) in dic.items():
                dic[k] = {str(int(k1)): v1 for (k1,v1) in v.items()}
                dic[k]["cnt"] = sum(v.values())
                bad_rate = round(dic[k].get("1", 0)/ dic[k]["cnt"], 5)
                dic[k]["bad_rate"] = bad_rate
            #利用定义的函数进行合并
            dic = self.combine_box_char(dic)
            #对每个特征计算WOE值和IV值
            for (k,v) in dic.items():
                a = v.get("0", 1) / good + 1e-10  
                b = v.get("1", 1) / bad + 1e-10
                dic[k]["GOOD"] = v.get("0", 0)
                dic[k]["BAD"] = v.get("1", 0)
                dic[k]["woe"] = round(math.log(a / b), 5)
                
            '''
            按照分箱后的点进行分割， 
            计算得到每一个特征值的WOE值， 
            将原始特征名加上'_woe'后缀，并赋予WOE值。
            '''
            for (klis,v) in dic.items():
                # 训练集进行替换
                for k in kils.split(","):
                    self.devf.loc[self.devf[col]==k,
                                                    "%s_woe" % col] = v["woe"]
                    #测试集进行替换
                    if not isinstance(self.valf, str):
                        self.valf.loc[self.valf[col]==k,
                                                    "%s_woe" % col] = v["woe"]
                    #对跨时间验证集进行替换
                    if not isinstance(self.offf, str):
                        self.offf.loc[self.offf[col]==k,
                                                    "%s_woe" % col] = v["woe"]
        return {"dev": self.devf, "val": self.valf, "off": self.offf} #返回新字典，包括三个数据集
  

基于负样本占比差异最大化的分箱原则

实施两种分箱策略。 

1.不同箱之间负样本占比差异最大化。 

2.每一箱的样本量不能过少。(不小于整体样本的5%)
        
箱的总数在5箱之内（不超过10箱)。通过控制划分后的总箱数，来迭代进行箱的合并。


In [15]:
    '''
    策略一：不同箱之间负样本占比差异最大化
    '''
    
    def combine_box_char(self, dic): 
        while len(dic) >= 10:
            #k:特征值，v["bad_rate"]：特征值对应的负样本占比
            bad_rate_dic = {k: v["bad_rate"] 
                                             for (k, v) in dic.items()} 
            #按样本占比排序，因为离散型变量，所以是无序的。可以直接写成递增形式
            bad_rate_sorted = sorted(bad_rate_dic.items(), key=lambda x: x[1])
            #计算每两箱之间的负样本占比差值。将差值最小的两箱进行合并
            bad_rate = [bad_rate_sorted[i+1][1] - bad_rate_sorted[i][1]
                        for i in range(len(bad_rate_sorted)-1)]
            min_rate_index = bad_rate.index(min(bad_rate))  
            #k1 k2 分别为差值最小的两箱的key
            k1 = bad_rate_sorted[min_rate_index][0]
            k2 = bad_rate_sorted[min_rate_index+1][0]
            #得到重新划分后的字典，箱的个数比之前少1个
            dic["%s,%s" % (k1, k2)] = dict()
            dic["%s,%s" % (k1, k2)]["0"] = dic[k1].get("0",0) + dic[k2].get("0",0)
            dic["%s,%s" % (k1, k2)]["1"] = dic[k1].get("1",0) + dic[k2].get("1",0)
            dic["%s,%s" % (k1, k2)]["cnt"] = dic[k1].get("cnt",0) + dic[k2].get("cnt",0)
            dic["%s,%s" % (k1, k2)]["bad_rate"] = round(dic["%s,%s" % (k1, k2)] / dic["%s,%s" % (k1, k2)]["cnt"],5)
            
            #删除旧字典
            del dic[k1], dic[k2]
       
        
            
            
            
            
#结束循环后，箱的个数应该少于10

'''
策略二：每一箱的样本量不能过少
将样本数量少的箱合并至其他箱中，以保证每一箱的样本数量不要太少。
'''
    #记录当前样本最少的箱的个数
        min_cnt = min([v["cnt"] for v in dic.values()])
        while len(dic) > 5 and min_cnt < self.nrows * 0.05:
            
            min_key = [k for (k,v) in dic.items() 
                                            if v["cnt"] == min_cnt][0]
            bad_rate_dic = {k: v["bad_rate"]   #每两箱之间的负样本占比差值             
                                    for (k,v) in dic.items()}
            bad_rate_sorted = sorted(bad_rate_dic.items(), key=lambda x: x[1])
            keys = [k[0] for k in bad_rate_sorted]
            min_index = keys.index(min_key)
            

            #同样想保持合并后箱之间的负样本占比差异最大化。 
            #由于箱的位置不同，按照三种不同情况进行分类讨论。
            
            
            if min_index == 0:
                k1, k2 = keys[:2]
            elif min_index == len(dic) - 1:
                k1, k2 = keys[-2:]
            else:
                bef_bad_rate = dic[min_key]["bad_rate"] - dic[keys[min_index - 1]]["bad_rate"]
                aft_bad_rate = dic[keys[min_index - 1]]["bad_rate"] - dic[min_key]["bad_rate"]
                if bef_bad_rate < aft_bad_rate:
                    k1,k2 = keys[min_index - 1], min_key
                else:
                    k1,k2 = min_key,keys[min_index + 1]
                
            
        #得到重新划分后的字典，箱的个数比之前少一。  
            dic["%s,%s" % (k1, k2)] = dict()
            dic["%s,%s" % (k1, k2)]["0"] = dic[k1].get("0",0) + dic[k2].get("0",0)
            dic["%s,%s" % (k1, k2)]["1"] = dic[k1].get("1",0) + dic[k2].get("1",0)
            dic["%s,%s" % (k1, k2)]["cnt"] = dic[k1].get("cnt",0) + dic[k2].get("cnt",0)
            dic["%s,%s" % (k1, k2)]["bad_rate"] = round(dic["%s,%s" % (k1, k2)] / dic["%s,%s" % (k1, k2)]["cnt"],5)
            
        #删除旧字典
            del dic[k1], dic[k2]
            min_cnt = min([v["cnt"] for v in dic.values()])
        return dic
                

IndentationError: unexpected indent (<ipython-input-15-47b77aa1bd28>, line 41)